# Important Data Structures in Physics
In every domain there are algorithms and data structures that seem to pop up over
and over again. We have already seen the most important data structure for most 
scientific computing: arrays. Since these received an entire chapter, we will 
forgo talking about them here. Instead, we will cover:

- Hash tables : Useful whenever you want to create associations between data
- Data frames : Similar to structured arrays or tables, but with added 
  capabilities that make them invaluable for experimental data
- B-trees : Useful for managing array chunks and hierarchies
- K-d trees : Space-partitioning data structures that are useful when trying to reason about the
  closest neighboring points in space


##Hash Tables
Hash tables are everywhere in software development. The most common one that we
have used so far is the Python dictionary.

In [1]:
hash('') == hash(0) == hash(0.0) == hash(False) == 0

True

## Data Frames
A relative newcomer, the data frame is a must-have data structure for data analysis. It
is particularly useful to experimentalists.

In [2]:
import pandas as pd

### Series
The Series class in pandas is effectively a one-dimensional NumPy array with an
optional associated index. 

A series may be created using array-like mechanisms, and they share the same 
primitive dtype system that NumPy arrays use. The following example creates 
a series of 64-bit floats:

In [3]:
pd.Series([42, 43, 44], dtype='f8')

0    42
1    43
2    44
dtype: float64

The following shows a series s with various particle names used as the index and the
values representing the number of the associated particle that a detector has seen:

In [4]:
s = pd.Series([42, 43, 44], 
      index=["electron", 
             "proton", 
             "neutron"])

In [5]:
s

electron    42
proton      43
neutron     44
dtype: int64

In [6]:
s['electron']

42

In [7]:
# inclusive bounds
s['electron':'proton']

electron    42
proton      43
dtype: int64

In [8]:
# integer indexing still OK
s[1:]

proton     43
neutron    44
dtype: int64

Series may also be created from dictionaries.

In [9]:
t = pd.Series({'electron': 6, 
              'neutron': 28, 
              'proton': 496, 
              'neutrino': 8128})

In [10]:
t

electron       6
neutrino    8128
neutron       28
proton       496
dtype: int64

Arithmetic can be peformed on series, on the index. 
Can you explain why neutrino is NaN in the following case?

In [11]:
s + t

electron     48
neutrino    NaN
neutron      72
proton      539
dtype: float64

### Exercise: Create a DataFrame from a dict

We can create a data frame from a dictionary of arrays, lists, or series. The
keys of the dictionary become the column names. 

1) Reusing the definitions of s and t from before, create a data frame called df.

In [12]:
df = pd.DataFrame({'S': s, 'T': t})

In [13]:
df

,S,T
electron,42,6
neutrino,NaN,8128
neutron,44,28
proton,43,496


2) Slice every other element

In [14]:
df[::2]

,S,T
electron,42,6
neutron,44,28


3) Add a new index to the data frame and value to S.

In [15]:
dg = df.append(pd.DataFrame({'S': [-8128]}, index=['antineutrino']))
dg

,S,T
electron,42,6
neutrino,NaN,8128
neutron,44,28
proton,43,496
antineutrino,-8128,NaN


4) Delete the neutron index.

In [16]:
dh = df.drop('neutron')
dh

,S,T
electron,42,6
neutrino,NaN,8128
proton,43,496


You may also easily transpose the rows and columns via the T attribute, as seen here:

In [17]:
df.T

,electron,neutrino,neutron,proton
S,42,NaN,44,43
T,6,8128,28,496


Boolean logic can be applied to a whole data frame. This creates a 'Mask.'

In [18]:
df < 42

,S,T
electron,False,True
neutrino,False,False
neutron,False,True
proton,False,False


### Exercise: Using a Mask
1) Create a mask called mymask that returns true for even numbers, false for odd.
2) Call mask() on the df, providing mymask as an argument. 
3) What is the result? Why?
4) Discuss with your neighbor... How might masks be useful in analysis?

In [19]:
mymask = df % 2 == 0
mymask
df.mask(mymask)

,S,T
electron,NaN,NaN
neutrino,NaN,NaN
neutron,NaN,NaN
proton,43,NaN


In [20]:
# accessing a single column 
# will return a series
df['T']

electron       6
neutrino    8128
neutron       28
proton       496
Name: T, dtype: int64

In [21]:
# setting a name to a series
# or expression will add a 
# column to the frame
df['small'] = df['T'] < 100
df

,S,T,small
electron,42,6,True
neutrino,NaN,8128,False
neutron,44,28,True
proton,43,496,False


In [22]:
# deleting a column will
# remove it from the frame
del df['small']
df

,S,T
electron,42,6
neutrino,NaN,8128
neutron,44,28
proton,43,496


## B-Trees
Let’s take a quick look at the blist package.
This package has a sorteddict type that implements a traditional B-tree data
structure. Creating a sorteddict is similar to creating a Python dictionary.

###Exercise: Create a B-Tree 

1) import sorteddict from blist. You may need to pip install blist.

In [31]:
#!pip install blist
from blist import sorteddict

2) create a new B-tree with some initial values

In [32]:
b = sorteddict(first="Albert", 
               last="Einstein",
               birthday=[1879, 3, 14])
b

sorteddict({'birthday': [1879, 3, 14], 'first': 'Albert', 'last': 'Einstein'})

3) and add a value after it has been created

In [33]:
b['died'] = [1955, 4, 18]
b

sorteddict({'birthday': [1879, 3, 14], 'died': [1955, 4, 18], 'first': 'Albert', 'last': 'Einstein'})

In [34]:
list(b.keys())

['birthday', 'died', 'first', 'last']

## K-D Trees
A k-d tree, or k-dimensional tree, is another tree data structure. This one excels at
finding the nearest neighbor for points in a k-dimensional space.

In [35]:
class Node(object):
    
    def __init__(self, point, left=None, right=None):
        self.point = point
        self.left = left
        self.right = right
        
    def __repr__(self):
        isleaf = self.left is None and self.right is None
        s = repr(self.point)
        if not isleaf:
            s = "[" + s + ":"
        if self.left is not None:
            s += "\n  left = " + "\n  ".join(repr(self.left).split('\n'))
        if self.right is not None:
            s += "\n  right = " + "\n  ".join(repr(self.right).split('\n'))
        if not isleaf:
            s += "\n  ]"
        return s


def kdtree(points, depth=0):
    if len(points) == 0:
        return None
    k = len(points[0])
    a = depth % k
    points = sorted(points, key=lambda x: x[a])
    i = int(len(points) / 2)  # middle index, rounded down
    node_left = kdtree(points[:i], depth + 1)
    node_right = kdtree(points[i+1:], depth + 1)
    node = Node(points[i], node_left, node_right)
    return node

In [36]:
points = [(1, 2), (3, 2), 
          (5, 5), (2, 1), 
          (4, 3), (1, 5)]
root = kdtree(points)
print(root)

[(3, 2):
  left = [(1, 2):
    left = (2, 1)
    right = (1, 5)
    ]
  right = [(5, 5):
    left = (4, 3)
    ]
  ]


In [37]:
from scipy.spatial import KDTree
tree = KDTree(points)

In [38]:
tree.data

array([[1, 2],
       [3, 2],
       [5, 5],
       [2, 1],
       [4, 3],
       [1, 5]])

In [39]:
# query() defaults to only the closest point
dist, idx = tree.query([(4.5, 1.25)])

In [40]:
dist

array([ 1.67705098])

In [41]:
idx 

array([1])

In [42]:
# fancy index by idx to get the point
tree.data[idx]

array([[3, 2]])

## Data Structures Wrap-up


You should now be familiar with the following
concepts:
- Hash tables require a good hash function, which is provided to you by Python.
- Resizing a hash table can be expensive.
- Hash collisions will happen.
- Series are like NumPy arrays but with more generic indexing capabilities.
- Data frames are like tables with series as columns.
- Data frames handle missing data through NaN values.
- B-trees can be used to organize chunks of an array.
- You may rotate B-trees to change their structure without altering their performance.
- A binary search tree is a B-tree with only one piece of data per node.
- K-d trees are a variant of the binary search tree that are organized by points in kdimensional 
  space.
- K-d trees are exceptionally useful for problems involving geometry.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()